# Choosing Appropriate Exposure Times and Dark-Frame Rules

In [17]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import time

In [18]:
from bluesky_tutorial_utils import setup_data_saving
from bluesky import RunEngine
from bluesky.plans import count

RE = RunEngine()
catalog = setup_data_saving(RE)

We will import some simulated devices from a Python script in the current directory, [simulated_hardware.py](./simulated_hardware.py). The objects present a programmatic interface that looks like real hardware, but their readings from come functions defined in another script, [generate_data.py](./generate_data.py), instead of from actual measurements.

## Manual Walk-through

First we'll take an exposure with the shutter open, then with it closed, and subtract the two, visualizing the results as we go.

In [6]:
from simulated_hardware import detector, shutter, load_sample, unload_sample, light, time_travel, current_time, _history, history_reset

In [13]:
history_reset()

In [14]:
_history

{'sample': [(1, 11)],
 'light': [(2, 10)],
 'image': [array([[1.92874985e-20, 4.18765668e-20, 8.98081999e-20, ...,
          1.90262788e-19, 8.98081999e-20, 4.18765668e-20],
         [4.18765668e-20, 9.09043553e-20, 1.94957132e-19, ...,
          4.13039338e-19, 1.94957132e-19, 9.09043553e-20],
         [8.98081999e-20, 1.94957132e-19, 4.18101915e-19, ...,
          8.85723023e-19, 4.18101915e-19, 1.94957132e-19],
         ...,
         [1.90262788e-19, 4.13039338e-19, 8.85723023e-19, ...,
          1.87658804e-18, 8.85723023e-19, 4.13039338e-19],
         [8.98081999e-20, 1.94957132e-19, 4.18101915e-19, ...,
          8.85723023e-19, 4.18101915e-19, 1.94957132e-19],
         [4.18765668e-20, 9.09043553e-20, 1.94957132e-19, ...,
          4.13039338e-19, 1.94957132e-19, 9.09043553e-20]])],
 'decay_a': 1000,
 'panel_amp': 2000,
 'panel_oset': 1,
 'panel_wid': 32,
 'noise': 50}

In [8]:
time_travel(1)
RE(load_sample(1))

time_travel(2)
RE(light(True))

time_travel(10)
RE(light(False))

time_travel(11)
RE(load_sample(0))

current time is 1
current time is 2
current time is 10
current time is 11


()

In [6]:
_history['decay_a'] = 1000

In [9]:
_history

{'sample': [(1, 11)],
 'light': [(2, 10)],
 'image': [array([[1.92874985e-20, 4.18765668e-20, 8.98081999e-20, ...,
          1.90262788e-19, 8.98081999e-20, 4.18765668e-20],
         [4.18765668e-20, 9.09043553e-20, 1.94957132e-19, ...,
          4.13039338e-19, 1.94957132e-19, 9.09043553e-20],
         [8.98081999e-20, 1.94957132e-19, 4.18101915e-19, ...,
          8.85723023e-19, 4.18101915e-19, 1.94957132e-19],
         ...,
         [1.90262788e-19, 4.13039338e-19, 8.85723023e-19, ...,
          1.87658804e-18, 8.85723023e-19, 4.13039338e-19],
         [8.98081999e-20, 1.94957132e-19, 4.18101915e-19, ...,
          8.85723023e-19, 4.18101915e-19, 1.94957132e-19],
         [4.18765668e-20, 9.09043553e-20, 1.94957132e-19, ...,
          4.13039338e-19, 1.94957132e-19, 9.09043553e-20]])],
 'decay_a': 1000,
 'panel_amp': 2000,
 'panel_oset': 1,
 'panel_wid': 32,
 'noise': 50}

In [7]:
time_travel

<function simulated_hardware.time_travel(t)>

In [8]:
time_travel(8)

In [10]:
RE(count([detector]))

current time is 11
amp 2000


('f18a82b6-8a39-4802-be90-06001424c144',)

In [11]:
tlist = np.linspace(0,20,21)
intensity_list = []
t0 = time.time()
for t in tlist:
    time_travel(t)
    RE(count([detector]))
    data = catalog[-1].primary.read()
    #this next line is a bit of a piece of work... trying to make a list of total intensities on detector
    #first grabbing image from xarray, then turing into numpy array, then summing, then appending to list.
    intensity_list.append(np.sum(np.array(data.detector_image[0]))) 
print ('this took '+str(time.time()-t0))

current time is 0.0
amp 2000
current time is 1.0
amp 2000
current time is 2.0
amp 2000
current time is 3.0
amp 2000
current time is 4.0
amp 2000
current time is 5.0
amp 2000
current time is 6.0
amp 2000
current time is 7.0
amp 2000
current time is 8.0
amp 2000
current time is 9.0
amp 2000
current time is 10.0
amp 2000
current time is 11.0
amp 2000
current time is 12.0
amp 2000
current time is 13.0
amp 2000
current time is 14.0
amp 2000
current time is 15.0
amp 2000
current time is 16.0
amp 2000
current time is 17.0
amp 2000
current time is 18.0
amp 2000
current time is 19.0
amp 2000
current time is 20.0
amp 2000
this took 5.920114994049072


In [8]:
def custom_plan():
    for t in tlist:
        time_travel(t)
        yield from count([detector])
t0 = time.time()
        
uids = RE(custom_plan())
print ("this part took "+str(time.time()-t0))

intensity_list = []

t1 = time.time()
for uid in uids:
    data = catalog[uid].primary.read()
    intensity_list.append(np.sum(data.detector_image[0].values))
    
print ('but this part took '+str(time.time()-t1))

current time is 0.0
current time is 1.0
current time is 2.0
current time is 3.0
current time is 4.0
current time is 5.0
current time is 6.0
current time is 7.0
current time is 8.0
current time is 9.0
current time is 10.0
current time is 11.0
current time is 12.0
current time is 13.0
current time is 14.0
current time is 15.0
current time is 16.0
current time is 17.0
current time is 18.0
current time is 19.0
current time is 20.0
this part took 3.8844656944274902
but this part took 1.7046384811401367


In [6]:
_history

{'sample': [(1, 11)],
 'light': [(2, 10)],
 'image': [array([[1.92874985e-20, 4.18765668e-20, 8.98081999e-20, ...,
          1.90262788e-19, 8.98081999e-20, 4.18765668e-20],
         [4.18765668e-20, 9.09043553e-20, 1.94957132e-19, ...,
          4.13039338e-19, 1.94957132e-19, 9.09043553e-20],
         [8.98081999e-20, 1.94957132e-19, 4.18101915e-19, ...,
          8.85723023e-19, 4.18101915e-19, 1.94957132e-19],
         ...,
         [1.90262788e-19, 4.13039338e-19, 8.85723023e-19, ...,
          1.87658804e-18, 8.85723023e-19, 4.13039338e-19],
         [8.98081999e-20, 1.94957132e-19, 4.18101915e-19, ...,
          8.85723023e-19, 4.18101915e-19, 1.94957132e-19],
         [4.18765668e-20, 9.09043553e-20, 1.94957132e-19, ...,
          4.13039338e-19, 1.94957132e-19, 9.09043553e-20]])],
 'decay_a': 1000,
 'panel_oset': 1,
 'panel_wid': 100,
 'panel_amp': 10,
 'noise': 10}

In [29]:
plt.figure()
_history['panel_amp'] = 2000
_history['panel_oset'] = 1
_history['panel_wid'] = 32
_history['noise'] = 50

time_travel(1)
RE(count([detector]))
data = catalog[-1].primary.read()
dark_im = data.detector_image[0]

time_travel(5)
RE(count([detector]))
data = catalog[-1].primary.read()
light_im = data.detector_image[0]

plt.imshow(light_im-dark_im,vmin=0)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

current time is 1
amp 2000
current time is 5
amp 2000


In [37]:
from utils import simple_integration

intensity = simple_integration((light_im - dark_im).clip(0), num_bins=256)

/home/dan/.local/lib/python3.7/site-packages/scipy/ndimage/measurements.py:639: RuntimeWarning: invalid value encountered in true_divide
  return sum / numpy.asanyarray(count).astype(numpy.float)


In [38]:
plt.figure()
plt.plot(np.nan_to_num(intensity))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
t0 = time.time()

In [40]:
%%time
intensity_list
print (time.time()-t0)

0.012636661529541016
CPU times: user 80 µs, sys: 41 µs, total: 121 µs
Wall time: 125 µs


In [25]:
np.sum(data.detector_image[0].values)

74632.51817059379

In [7]:
plt.figure()
plt.plot(tlist, intensity_list)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
data = catalog[-1].primary.read()
data

<xarray.Dataset>
Dimensions:         (dim_0: 512, dim_1: 512, time: 1)
Coordinates:
  * time            (time) float64 1.59e+09
Dimensions without coordinates: dim_0, dim_1
Data variables:
    detector_image  (time, dim_0, dim_1) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    seq_num         (time) int64 1
    uid             (time) <U36 '075628f1-f9e5-4032-93eb-0b90fd67646a'

In [17]:
np.sum(np.array(data.detector_image[0]))

1246930695.090101

In [8]:
plt.figure()
plt.imshow(data.detector_image[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
axes[0].imshow(data["detector_image"].mean("time"))

In [ ]:
fig

In [ ]:
RE(light(False))

In [ ]:
time_travel(2000)

In [ ]:
RE(count([detector]))

In [ ]:
dark_data = catalog[-1].primary.read()
axes[1].imshow(dark_data["detector_image"].mean("time"))
fig

In [ ]:
subtracted = data["detector_image"].mean("time") - dark_data["detector_image"].mean("time")
axes[2].imshow(subtracted)
fig

Compute $I(q)$

In [ ]:
from utils import simple_integration

intensity = simple_integration(data["detector_image"].squeeze() - dark_data["detector_image"].squeeze())

These images were simulated based on a "ground truth" $I(q)$. Let's plot our computed $I(q)$ with the ground truth to check that they correspond closely.

In [ ]:
plt.figure()

In [ ]:
from simulated_hardware import intensities, x
plt.plot(x, intensities[1], "--", label="ground truth")
plt.plot(x[:len(intensity)], intensity, label="calculated")
plt.legend()
plt.gcf()

## Automatically capture dark frames

Use the utility [bluesky-darkframes](https://blueskyproject.io/bluesky-darkframes) to automatically capture darkframes as part of the data in every Run. This solves two problems for us:

1. We don't need to remember to take dark frames. They will be automatically taken for us.
2. Each Run is self-contained. Instead of having a "dark" Run and a "light" Run, every Run contains both light and dark together.

In [ ]:
from automatic_darkframes import setup_automatic_darkframes

dfp = setup_automatic_darkframes(RE)

In [ ]:
RE(count([detector]))

In [ ]:
data = catalog[-1].primary.read()
dark_data = catalog[-1].dark.read()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

In [ ]:
data["detector_image"].mean("time").plot(ax=axes[0])
dark_data["detector_image"].mean("time").plot(ax=axes[1])
(data["detector_image"].mean("time") - dark_data["detector_image"]).mean("time").plot(ax=axes[2])
fig.tight_layout()
fig